In [1]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from pandas_profiling import ProfileReport

sns.set()

# Context
The data we will be using through the pratical classes comes from a small relational database whose schema can be seen below:
![alt text](../figures/schema.png "Relation database schema")

# Reading the Data

In [2]:
# path to database
my_path = os.path.join("..", "data", "datamining.db")

# connect to the database
conn = sqlite3.connect(my_path)

# the query
query = """
select
    age, 
    income, 
    frq, 
    rcn, 
    mnt, 
    clothes, 
    kitchen, 
    small_appliances, 
    toys, 
    house_keeping,
    dependents, 
    per_net_purchase,
    g.gender, 
    e.education, 
    m.status, 
    r.description
from customers as c
    join genders as g on g.id = c.gender_id
    join education_levels as e on e.id = c.education_id
    join marital_status as m on m.id = c.marital_status_id
    join recommendations as r on r.id = c.recommendation_id
order by c.id;
"""

df = pd.read_sql_query(query, conn)

## Make a copy of your original dataset

why?

In [3]:
df_original = df.copy()

# Metadata
- *id* - The unique identifier of the customer
- *age* - The year of birht of the customer
- *income* - The income of the customer
- *frq* - Frequency: number of purchases made by the customer
- *rcn* - Recency: number of days since last customer purchase
- *mnt* - Monetary: amount of € spent by the customer in purchases
- *clothes* - Number of clothes items purchased by the customer
- *kitchen* - Number of kitchen items purchased by the customer
- *small_appliances* - Number of small_appliances items purchased by the customer
- *toys* - Number of toys items purchased by the customer
- *house_keeping* - Number of house_keeping items purchased by the customer
- *dependents* - Binary. Whether or not the customer has dependents
- *per_net_purchase* - Percentage of purchases made online
- *education* - Education level of the customer
- *status* - Marital status of the customer
- *gender* - Gender of the customer
- *description* - Last customer's recommendation description

## Problems:
- Duplicates?
- Data types?
- Missing values?
- Strange values?
- Descriptive statistics?

### Take a closer look and point out possible problems:

(hint: a missing values in pandas is represented with a NaN value)

In [4]:
# replace "" by nans
df.replace("", np.nan, inplace=True)

In [5]:
# check dataset data types again
df.dtypes

age                   int64
income              float64
frq                   int64
rcn                   int64
mnt                   int64
clothes               int64
kitchen               int64
small_appliances      int64
toys                  int64
house_keeping         int64
dependents          float64
per_net_purchase      int64
gender               object
education            object
status               object
description          object
dtype: object

In [6]:
# check descriptive statistics again
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
age,8998.0,NaN,NaN,NaN,1966.05968,17.296552,1936.0,1951.0,1966.0,1981.0,1996.0
income,8952.0,NaN,NaN,NaN,69963.550827,27591.556226,10000.0,47741.0,70030.5,92218.0,140628.0
frq,8998.0,NaN,NaN,NaN,19.848077,10.903435,3.0,10.0,17.0,28.0,59.0
rcn,8998.0,NaN,NaN,NaN,62.469771,69.761802,0.0,26.0,53.0,79.0,549.0
mnt,8998.0,NaN,NaN,NaN,622.162814,646.768205,6.0,63.0,383.0,1076.0,3052.0
clothes,8998.0,NaN,NaN,NaN,50.446655,23.422249,1.0,33.0,51.0,69.0,99.0
kitchen,8998.0,NaN,NaN,NaN,7.039675,7.848139,0.0,2.0,4.0,10.0,75.0
small_appliances,8998.0,NaN,NaN,NaN,28.524116,12.586437,1.0,19.0,28.0,37.0,74.0
toys,8998.0,NaN,NaN,NaN,7.036897,7.924422,0.0,2.0,4.0,10.0,62.0
house_keeping,8998.0,NaN,NaN,NaN,6.929984,7.882655,0.0,2.0,4.0,9.0,77.0


In [7]:
# Define metric and non-metric features. Why?
non_metric_features = ["education", "status", "gender", "dependents", "description"]
metric_features = df.columns.drop(non_metric_features).to_list()

## Fill missing values (Data imputation)

How can we fill missing values?

In [8]:
# Creating a copy to apply central tendency measures imputation
df_central = df.copy()

In [9]:
# count of missing values
df_central.isna().sum()

age                   0
income               46
frq                   0
rcn                   0
mnt                   0
clothes               0
kitchen               0
small_appliances      0
toys                  0
house_keeping         0
dependents          282
per_net_purchase      0
gender                0
education            47
status              177
description           0
dtype: int64

In [16]:
medians = df_central[metric_features].median()   #we cannot do median in non metric features
medians

age                  1966.0
income              70030.5
frq                    17.0
rcn                    53.0
mnt                   383.0
clothes                51.0
kitchen                 4.0
small_appliances       28.0
toys                    4.0
house_keeping           4.0
per_net_purchase       45.0
dtype: float64

In [15]:
modes = df_central[non_metric_features].mode()
modes

,education,status,gender,dependents,description
0,Graduation,Married,M,1.0,OK nice!


In [17]:
#OR
#input_values = pd.concat([medians, modes])
#df_central.fillna(input_values, inplace=True)

df_central.fillna(df_central.median(), inplace=True)
df_central.fillna(modes, inplace=True)
df_central.isna().sum()  # checking how many NaNs we still have

C:\Users\sofia\AppData\Local\Temp\ipykernel_20848\2302810431.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_central.fillna(df_central.median(), inplace=True)


age                   0
income                0
frq                   0
rcn                   0
mnt                   0
clothes               0
kitchen               0
small_appliances      0
toys                  0
house_keeping         0
dependents            0
per_net_purchase      0
gender                0
education            47
status              177
description           0
dtype: int64

In [18]:
# Creating new df copy to explore neighbordhood imputation
df_neighbors = df.copy()

In [19]:
# Seeing rows with NaNs
nans_index = df_neighbors.isna().any(axis=1)
df_neighbors[nans_index]

,age,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description
3,1955,93571.0,26,10,888,60,10,19,6,5,1.0,35,F,Master,NaN,OK nice!
61,1968,NaN,14,90,184,95,1,3,1,0,1.0,59,F,2nd Cycle,Married,Meh...
67,1981,60457.0,9,73,63,37,2,51,6,3,NaN,57,M,PhD,Married,Meh...
70,1944,116259.0,28,35,1279,17,6,54,3,20,0.0,11,M,Graduation,NaN,OK nice!
73,1967,75274.0,16,67,263,88,3,6,1,2,NaN,66,M,Graduation,Single,Kind of OK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8932,1959,NaN,20,71,716,67,5,25,3,1,0.0,27,M,Graduation,Married,Meh...
8942,1950,92746.0,34,98,1399,50,5,36,5,4,NaN,28,M,Master,Married,Take my money!!
8964,1978,44661.0,6,49,33,18,18,35,16,13,1.0,42,M,1st Cycle,NaN,OK nice!
8972,1976,NaN,17,9,254,70,1,28,1,1,1.0,59,M,Graduation,Divorced,Meh...


In [20]:
# KNNImputer - only works for numerical varaibles
imputer = KNNImputer(n_neighbors=5, weights="uniform")
df_neighbors[metric_features] = imputer.fit_transform(df_neighbors[metric_features])

In [23]:
# See rows with NaNs imputed

#df_neighbors.loc[nans_index, metric_features]

nans_index = df["income"].isna()
df_neighbors.loc[nans_index]      #averages including new neighbors

,age,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description
61,1968.0,63143.0,14.0,90.0,184.0,95.0,1.0,3.0,1.0,0.0,1.0,59.0,F,2nd Cycle,Married,Meh...
703,1988.0,40386.2,10.0,77.0,39.0,24.0,5.0,63.0,4.0,4.0,1.0,68.0,F,2nd Cycle,Together,OK nice!
793,1956.0,75781.8,15.0,25.0,338.0,72.0,1.0,25.0,0.0,1.0,1.0,48.0,M,Master,Single,OK nice!
852,1956.0,80303.8,21.0,95.0,596.0,69.0,4.0,20.0,2.0,6.0,1.0,33.0,M,Graduation,Together,OK nice!
1197,1974.0,65140.0,20.0,10.0,395.0,67.0,1.0,28.0,2.0,2.0,1.0,56.0,F,Graduation,Together,Meh...
1280,1949.0,69257.8,8.0,81.0,122.0,76.0,3.0,18.0,2.0,1.0,1.0,38.0,M,Master,NaN,Meh...
1614,1978.0,48645.8,7.0,55.0,51.0,70.0,4.0,20.0,1.0,5.0,1.0,41.0,M,Graduation,Married,OK nice!
1669,1987.0,60107.6,21.0,5.0,482.0,39.0,3.0,43.0,8.0,7.0,1.0,36.0,F,PhD,Married,Kind of OK
1917,1951.0,84048.2,19.0,24.0,593.0,49.0,10.0,30.0,10.0,1.0,1.0,23.0,M,Graduation,Married,Meh...
1924,1948.0,99651.8,31.0,86.0,1468.0,46.0,3.0,40.0,6.0,5.0,0.0,11.0,F,Master,Divorced,OK nice!


In [24]:
# let's keep the central imputation
df = df_central.copy()

## An overview of our previous data exploration

You can also explore this dataset using the exported `pandas-profiling` report.



![](../figures/exp_analysis/categorical_variables_frequecies.png)

![](../figures/exp_analysis/numeric_variables_histograms.png)

![](../figures/exp_analysis/numeric_variables_boxplots.png)

![](../figures/exp_analysis/pairwise_relationship_of_numerical_variables.png)

![](../figures/exp_analysis/correlation_matrix.png)

## Outlier removal

Why do we need to remove outliers? Which methods can we use?


Let's start by "manually" filtering the dataset's outliers

In [25]:
# This may vary from session to session, and is prone to varying interpretations.
# A simple example is provided below:

filters1 = (
    (df['house_keeping']<=50)
    &
    (df['kitchen']<=40)
    &
    (df['toys']<=35)
    &
    (df['education']!='OldSchool')
    &
    (df['status'] != "Whatever")
)

df_1 = df[filters1]

In [26]:
print('Percentage of data kept after removing outliers:', np.round(df_1.shape[0] / df_original.shape[0], 4))

Percentage of data kept after removing outliers: 0.9783


### Outlier removal using only the IQR method

Why should you use/not use this method?

In [28]:
q25 = df.quantile(.25)
q75 = df.quantile(.75)
iqr = (q75 - q25)

upper_lim = q75 + 1.5 * iqr
lower_lim = q25 - 1.5 * iqr

filters2 = []
for metric in metric_features:
    
    llim = lower_lim[metric]
    ulim = upper_lim[metric]
    filters2.append(df[metric].between(llim, ulim, inclusive='both'))

filters2 = pd.Series(np.all(filters2, 0))
df_2 = df[filters2]
print('Percentage of data kept after removing outliers:', np.round(df_2.shape[0] / df_original.shape[0], 4))

Percentage of data kept after removing outliers: 0.8242


What do you think about this percentage? It is too high (17,58%) We are removing too much of information

## Combining different outlier methods

More robust/ consistent outlier detection method:

In [29]:
df_3 = df[(filters1 | filters2)]
print('Percentage of data kept after removing outliers:', np.round(df_3.shape[0] / df_original.shape[0], 4))

#create a more agressive filter approach for outliers removal

Percentage of data kept after removing outliers: 0.9817


In [30]:
# Get the manual filtering version
df = df_1.copy()

## Feature Engineering

A reminder of our metadata:
- *id* - The unique identifier of the customer
- *age* - The year of birht of the customer
- *income* - The income of the customer
- *frq* - Frequency: number of purchases made by the customer
- *rcn* - Recency: number of days since last customer purchase
- *mnt* - Monetary: amount of € spent by the customer in purchases
- *clothes* - Number of clothes items purchased by the customer
- *kitchen* - Number of kitchen items purchased by the customer
- *small_appliances* - Number of small_appliances items purchased by the customer
- *toys* - Number of toys items purchased by the customer
- *house_keeping* - Number of house_keeping items purchased by the customer
- *dependents* - Binary. Whether or not the customer has dependents
- *per_net_purchase* - Percentage of purchases made online
- *education* - Education level of the customer
- *status* - Marital status of the customer
- *gender* - Gender of the customer
- *description* - Last customer's recommendation description

In [35]:
# Create the "birth_year" and change the "age" variable. 
# Also, create a new variable "spent_online": amount of € spent by the customer in online purchases

#df.rename(columns={"age":"birth_year"}, inplace=True)  #we should only run it once 
df["age"] = datetime.now().year - df["birth_year"]

df["spent_online"] = df["mnt"]*df["per_net_purchase"]/100

df

#detect outliers for example by comparing income with what I spent in the shop (mnt)

,birth_year,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description,age,spent_online
0,1946,90782.0,33,66,1402,37,5,44,10,3,0.0,19,M,Graduation,Together,Take my money!!,76,266.38
1,1936,113023.0,32,6,1537,55,1,38,4,2,0.0,9,F,PhD,Divorced,Take my money!!,86,138.33
2,1990,28344.0,11,69,44,32,19,24,1,24,1.0,59,M,Graduation,Married,Kind of OK,32,25.96
3,1955,93571.0,26,10,888,60,10,19,6,5,1.0,35,F,Master,NaN,OK nice!,67,310.80
4,1955,91852.0,31,26,1138,59,5,28,4,4,1.0,34,F,Graduation,Together,Take my money!!,67,386.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8992,1954,87399.0,25,1,837,56,8,27,8,1,1.0,47,M,Graduation,Married,Kind of OK,68,393.39
8993,1960,94367.0,28,1,896,68,5,21,3,4,1.0,55,F,1st Cycle,Single,Take my money!!,62,492.80
8994,1975,58121.0,12,6,61,53,6,28,7,6,1.0,71,M,2nd Cycle,Single,Meh...,47,43.31
8995,1986,54292.0,29,72,1011,41,11,36,1,11,0.0,31,M,Graduation,Together,Take my money!!,36,313.41


In [39]:
# How can we avoid having as many extreme values in 'rcn' (look at distribution)? (Discuss)

print((df['rcn']>100).value_counts()) #OR from collections import Counter ; Counter((df['rcn']>100))

df.loc[df['rcn']>100, "rcn"] = 100

False    8423
True      380
Name: rcn, dtype: int64


## Variable selection: Redundancy VS Relevancy

### Redundancy
We already saw our original correlation matrix:
![](../figures/exp_analysis/correlation_matrix.png)

In [41]:
# Drop variables according to their correlations

df.drop(columns = ["mnt", "age", "birth_year"], inplace = True) #bc it is more correlated with other features than frq; age and birth_year are removed due to the low significancy


,birth_year,income,frq,rcn,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description,spent_online
0,1946,90782.0,33,66,37,5,44,10,3,0.0,19,M,Graduation,Together,Take my money!!,266.38
1,1936,113023.0,32,6,55,1,38,4,2,0.0,9,F,PhD,Divorced,Take my money!!,138.33
2,1990,28344.0,11,69,32,19,24,1,24,1.0,59,M,Graduation,Married,Kind of OK,25.96
3,1955,93571.0,26,10,60,10,19,6,5,1.0,35,F,Master,NaN,OK nice!,310.80
4,1955,91852.0,31,26,59,5,28,4,4,1.0,34,F,Graduation,Together,Take my money!!,386.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8992,1954,87399.0,25,1,56,8,27,8,1,1.0,47,M,Graduation,Married,Kind of OK,393.39
8993,1960,94367.0,28,1,68,5,21,3,4,1.0,55,F,1st Cycle,Single,Take my money!!,492.80
8994,1975,58121.0,12,6,53,6,28,7,6,1.0,71,M,2nd Cycle,Single,Meh...,43.31
8995,1986,54292.0,29,72,41,11,36,1,11,0.0,31,M,Graduation,Together,Take my money!!,313.41


In [64]:
# Updating metric_features
#metric_features.remove("mnt")
#metric_features.remove("age")
#metric_features.remove("birth_year")
#metric_features.append("spent_online")
metric_features

ValueError: list.remove(x): x not in list

### Relevancy
Selecting variables based on the relevancy of each one to the task. Example: remove uncorrelated variables with the target, stepwise regression, use variables for product clustering, use variables for socio-demographic clustering, ...

Variables that aren't correlated with any other variable are often also not relevant. In this case we will not focus on this a lot since we don't have a defined task yet.

## Data Normalization

Why do we scale our data? Discuss!

In [65]:
df_minmax = df.copy()

In [66]:
# Use MinMaxScaler to scale the data (all values will be bet (0,1) but we don't have the certain regarding mean and standard deviation)
scaler = MinMaxScaler()
scaled_feat = scaler.fit_transform(df_minmax[metric_features])
scaled_feat

array([[0.61841259, 0.53571429, 0.66      , ..., 0.1865215 , 0.1865215 ,
        0.1865215 ],
       [0.78867471, 0.51785714, 0.06      , ..., 0.0959314 , 0.0959314 ,
        0.0959314 ],
       [0.14042931, 0.14285714, 0.69      , ..., 0.01643427, 0.01643427,
        0.01643427],
       ...,
       [0.36838197, 0.16071429, 0.06      , ..., 0.02870868, 0.02870868,
        0.02870868],
       [0.33906972, 0.46428571, 0.72      , ..., 0.21979328, 0.21979328,
        0.21979328],
       [0.88772698, 0.625     , 0.75      , ..., 0.52908717, 0.52908717,
        0.52908717]])

In [67]:
# See what the fit method is doing (notice the trailing underscore):
print("Parameters fitted:\n", scaler.data_min_, "\n", scaler.data_max_)

Parameters fitted:
 [1.00e+04 3.00e+00 0.00e+00 1.00e+00 0.00e+00 1.00e+00 0.00e+00 0.00e+00
 4.00e+00 2.73e+00 2.73e+00 2.73e+00] 
 [1.40628e+05 5.90000e+01 1.00000e+02 9.90000e+01 4.00000e+01 7.40000e+01
 3.50000e+01 5.00000e+01 8.80000e+01 1.41624e+03 1.41624e+03 1.41624e+03]


In [68]:
df_minmax[metric_features] = scaled_feat
df_minmax.head()

,birth_year,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description,age,spent_online
0,1946,0.618413,0.535714,0.66,1402,0.367347,0.125,0.589041,0.285714,0.06,0.0,0.178571,M,Graduation,Together,Take my money!!,76,0.186521
1,1936,0.788675,0.517857,0.06,1537,0.551020,0.025,0.506849,0.114286,0.04,0.0,0.059524,F,PhD,Divorced,Take my money!!,86,0.095931
2,1990,0.140429,0.142857,0.69,44,0.316327,0.475,0.315068,0.028571,0.48,1.0,0.654762,M,Graduation,Married,Kind of OK,32,0.016434
3,1955,0.639763,0.410714,0.10,888,0.602041,0.250,0.246575,0.171429,0.10,1.0,0.369048,F,Master,NaN,OK nice!,67,0.217947
4,1955,0.626604,0.500000,0.26,1138,0.591837,0.125,0.369863,0.114286,0.08,1.0,0.357143,F,Graduation,Together,Take my money!!,67,0.271799


In [69]:
# Figure out what characteristics the MinMaxed data have
df_minmax[metric_features].describe().round(2)

,income,frq,rcn,clothes,kitchen,small_appliances,toys,house_keeping,per_net_purchase,spent_online,spent_online,spent_online
count,8803.00,8803.00,8803.00,8803.00,8803.00,8803.00,8803.00,8803.00,8803.00,8803.00,8803.00,8803.00
mean,0.46,0.30,0.52,0.51,0.17,0.38,0.19,0.14,0.46,0.12,0.12,0.12
std,0.21,0.20,0.30,0.23,0.18,0.17,0.20,0.15,0.22,0.12,0.12,0.12
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,0.30,0.12,0.26,0.34,0.05,0.25,0.06,0.04,0.27,0.02,0.02,0.02
50%,0.46,0.27,0.53,0.52,0.10,0.37,0.11,0.08,0.49,0.10,0.10,0.10
75%,0.63,0.45,0.78,0.70,0.22,0.49,0.26,0.18,0.63,0.19,0.19,0.19
max,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [70]:
df_standard = df.copy()

In [71]:
# Use StandardScaler to scale the data
scaler = StandardScaler()
scaled_feat = scaler.fit_transform(df_standard[metric_features])
scaled_feat

array([[ 0.74371875,  1.19223742,  0.45855932, ...,  0.52318737,
         0.52318737,  0.52318737],
       [ 1.55972782,  1.10066049, -1.53477321, ..., -0.23656419,
        -0.23656419, -0.23656419],
       [-1.54709436, -0.82245511,  0.55822595, ..., -0.90328254,
        -0.90328254, -0.90328254],
       ...,
       [-0.45459393, -0.73087818, -1.53477321, ..., -0.8003408 ,
        -0.8003408 , -0.8003408 ],
       [-0.59507767,  0.82592969,  0.65789258, ...,  0.80222772,
         0.80222772,  0.80222772],
       [ 2.03445204,  1.65012209,  0.7575592 , ...,  3.39618155,
         3.39618155,  3.39618155]])

In [72]:
# See what the fit method is doing (notice the trailing underscore):
print("Parameters fitted:\n", scaler.mean_, "\n", scaler.var_)

Parameters fitted:
 [7.05113324e+04 1.99810292e+01 5.21972055e+01 5.11625582e+01
 6.78064296e+00 2.86818130e+01 6.58525503e+00 6.76746564e+00
 4.22500284e+01 1.78200987e+02 1.78200987e+02 1.78200987e+02] 
 [7.42879966e+08 1.19241603e+02 9.06030859e+02 5.27947561e+02
 5.34473954e+01 1.58535699e+02 4.75207050e+01 5.68909465e+01
 3.41706542e+02 2.84063841e+04 2.84063841e+04 2.84063841e+04]


In [73]:
df_standard[metric_features] = scaled_feat
df_standard.head()

,birth_year,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,dependents,per_net_purchase,gender,education,status,description,age,spent_online
0,1946,0.743719,1.192237,0.458559,1402,-0.616377,-0.243564,1.216589,0.495355,-0.499491,0.0,-1.257757,M,Graduation,Together,Take my money!!,76,0.523187
1,1936,1.559728,1.100660,-1.534773,1537,0.167012,-0.790702,0.740062,-0.375026,-0.632071,0.0,-1.798727,F,PhD,Divorced,Take my money!!,86,-0.236564
2,1990,-1.547094,-0.822455,0.558226,44,-0.833985,1.671418,-0.371835,-0.810217,2.284692,1.0,0.906124,M,Graduation,Married,Kind of OK,32,-0.903283
3,1955,0.846046,0.551199,-1.401884,888,0.384619,0.440358,-0.768941,-0.084899,-0.234331,1.0,-0.392205,F,Master,NaN,OK nice!,67,0.786742
4,1955,0.782976,1.009084,-0.870329,1138,0.341098,-0.243564,-0.054150,-0.375026,-0.366911,1.0,-0.446302,F,Graduation,Together,Take my money!!,67,1.238380


In [75]:
# Figure out what characteristics the Standardized data have
df_standard[metric_features].describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
income,8803.0,0.0,1.0,-2.22,-0.80,-0.01,0.80,2.57
frq,8803.0,-0.0,1.0,-1.56,-0.91,-0.18,0.73,3.57
rcn,8803.0,-0.0,1.0,-1.73,-0.87,0.03,0.86,1.59
clothes,8803.0,-0.0,1.0,-2.18,-0.75,0.04,0.82,2.08
kitchen,8803.0,0.0,1.0,-0.93,-0.65,-0.38,0.30,4.54
small_appliances,8803.0,0.0,1.0,-2.20,-0.77,-0.05,0.66,3.60
toys,8803.0,-0.0,1.0,-0.96,-0.67,-0.38,0.35,4.12
house_keeping,8803.0,0.0,1.0,-0.90,-0.63,-0.37,0.30,5.73
per_net_purchase,8803.0,-0.0,1.0,-2.07,-0.82,0.15,0.80,2.47
spent_online,8803.0,0.0,1.0,-1.04,-0.85,-0.23,0.51,7.35


**Important**: What if we had a training and validation/test set? Should we fit a Scaler in both? What about other Sklearn objects?

In [76]:
df = df_standard.copy()  #keep standard approach

## One-hot encoding

In [77]:
df_ohc = df.copy()

In [78]:
# First let's remove status=Whatever
df_ohc.loc[df_ohc['status'] == 'Whatever', 'status'] = df['status'].mode()[0]

In [82]:
# Use OneHotEncoder to encode the categorical features. Get feature names and create a DataFrame 
# with the one-hot encoded categorical features (pass feature names)
ohc = OneHotEncoder(sparse=False, drop="first")
ohc_feat = ohc.fit_transform(df_ohc[non_metric_features])
ohc_feat_names = ohc.get_feature_names_out()
ohc_df = pd.DataFrame(columns = ohc_feat_names, data = ohc_feat, index= df_ohc.index )
ohc_df

,education_2nd Cycle,education_Graduation,education_Master,education_PhD,education_nan,status_Married,status_Single,status_Together,status_Widow,status_nan,gender_M,dependents_1.0,description_Kind of OK,description_Meh...,description_OK nice!,description_Take my money!!
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8992,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
8993,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
8994,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
8995,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [83]:
# Reassigning df to contain ohc variables
df_ohc = pd.concat([df_ohc.drop(columns=non_metric_features), ohc_df], axis=1)
df_ohc.head()

,birth_year,income,frq,rcn,mnt,clothes,kitchen,small_appliances,toys,house_keeping,...,status_Single,status_Together,status_Widow,status_nan,gender_M,dependents_1.0,description_Kind of OK,description_Meh...,description_OK nice!,description_Take my money!!
0,1946,0.743719,1.192237,0.458559,1402,-0.616377,-0.243564,1.216589,0.495355,-0.499491,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1936,1.559728,1.100660,-1.534773,1537,0.167012,-0.790702,0.740062,-0.375026,-0.632071,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1990,-1.547094,-0.822455,0.558226,44,-0.833985,1.671418,-0.371835,-0.810217,2.284692,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
3,1955,0.846046,0.551199,-1.401884,888,0.384619,0.440358,-0.768941,-0.084899,-0.234331,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1955,0.782976,1.009084,-0.870329,1138,0.341098,-0.243564,-0.054150,-0.375026,-0.366911,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [84]:
df = df_ohc.copy()